# 토큰 분류
- [강좌링크](https://wikidocs.net/166830)

#### 다음과 같이 문장의 각 토큰에 레이블을 지정하는 모든 문제를 포함:
- NER(Named Entity Recognition): 개체명 인식
- POS(Part-Of-Speech tagging): 문장의 각 단어에 대한 특정 품사를 지정
- Chunking: 동일한 개체명 혹은 엔티티에 속한 토큰 찾기

# 데이터 준비
토큰 분류에 적합한 Reuters의 주요 뉴스 기사가 포함된 CoNLL-2003 데이터셋 사용

## CoNLL-2003 데이터셋

In [13]:
from datasets import load_dataset

raw_datasets = load_dataset("conll2003")

In [14]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

NER, POS, Chunking에 대한 레이블이 데이터셋에 포함되어있다. 입력된 텍스트가 문장이나 문서로 표현되지 않고 단어의 목록으로 표현된다. `token` 칼럼은 subword tokenization을 위해 여전히 토크나이저를 통과해야하는 pre-tokenization된 입력이라는 의미에서 word가 포함되어있다.

In [15]:
print(raw_datasets["train"][0]["tokens"])
print(raw_datasets["train"][0]["ner_tags"])

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']
[3, 0, 7, 0, 0, 0, 7, 0, 0]


In [16]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [17]:
label_names = ner_feature.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

이제 레이블을 디코딩함으로써 다음과 같은 결과를 볼 수 있다.

In [18]:
label_names_ner_tags = raw_datasets["train"].features["ner_tags"].feature.names
label_names_pos_tags = raw_datasets["train"].features["pos_tags"].feature.names
label_names_chunk_tags = raw_datasets["train"].features["chunk_tags"].feature.names

In [19]:
words = raw_datasets["train"][0]["tokens"]
ner_tags = raw_datasets["train"][0]["ner_tags"]
pos_tags = raw_datasets["train"][0]["pos_tags"]
chunk_tags = raw_datasets["train"][0]["chunk_tags"]

sentence = ""
ner_str = ""
pos_str = ""
chunk_str = ""

for word, ner, pos, chunk in zip(words, ner_tags, pos_tags, chunk_tags):
    ner_tag = label_names_ner_tags[ner]
    pos_tag = label_names_pos_tags[pos]
    chunk_tag = label_names_chunk_tags[chunk]
    max_length = max(len(word), len(ner_tag), len(pos_tag), len(chunk_tag))
    
    sentence += word + " " * (max_length - len(word) + 1)
    ner_str += ner_tag + " " * (max_length - len(ner_tag) + 1)
    pos_str += pos_tag + " " * (max_length - len(pos_tag) + 1)
    chunk_str += chunk_tag + " " * (max_length - len(chunk_tag) + 1)

print(sentence)
print(ner_str)
print(pos_str)
print(chunk_str)

EU    rejects German call to   boycott British lamb . 
B-ORG O       B-MISC O    O    O       B-MISC  O    O 
NNP   VBZ     JJ     NN   TO   VB      JJ      NN   . 
B-NP  B-VP    B-NP   I-NP B-VP I-VP    B-NP    I-NP O 


## 데이터 처리
텍스트를 토큰 ID로 변환해야 모델이 해당 입력을 이해할 수 있다. 토큰 분류 작업의 경우 이미 단어로 분할(pre-tokenization)된 입력이 존재한다는 것인데 토크나이저 API는 이를 매우 쉽게 처리할 수 있다. `bert-base-cased` 토크나이저 모델을 다운로드하고 fast tokenizer인지 확인한 후 pre-tokenized 입력을 토큰화하려면 `is_split_into_words=True`를 지정하여 `tokenizer`를 실행한다.

In [20]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

print(tokenizer.is_fast)

inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
print(inputs.tokens())
print(inputs.word_ids())

True
['[CLS]', 'EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'la', '##mb', '.', '[SEP]']
[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]


토큰과 일치되도록 레이블 확장하기
1. 특수 토큰의 레이블 ID는 -100(사용할 손실함수 cross entropy에서 무시되는 인덱스)
2. 위 토큰 식별자 리스트에서 첫번째 7에 해당하는 토큰의 레이블은 B-로, 두번째 7에 해당하는 토큰의 레이블은 I-로 시작한다.

In [21]:
# [CLS], [SEP] 2 개의 특수 토큰에 -100 할당하고 두 개의 토크으로 분할된 토큰에 대해 0 추가
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # 새로운 단어의 시작
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # 특수 토큰
            new_labels.append(-100)
        else:
            # 이전 토큰과 동일한 단어에 소속된 토큰
            label = labels[word_id]
            
            # 만약 레이블이 B-XXX면 I-XXX로 변경
            if label % 2 == 1:
                label += 1
            
            new_labels.append(label)
    return new_labels

print(ner_tags)
print(align_labels_with_tokens(ner_tags, inputs.word_ids()))

[3, 0, 7, 0, 0, 0, 7, 0, 0]
[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]


위와 비슷하지만 하나의 단어에 하나의 레이블만 지정하고 해당 단어의 다른 하위 토큰에 무시될 토큰을 입력하면 손실에 크게 기여하는 많은 하위 토큰으로 분할되는 긴 단어를 회피할 수 있다. 레이블을 입력 ID와 정렬할 수 있도록 변경한다.

In [22]:
def align_only_one_label(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # 새로운 단어의 시작 토큰
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        else:
            new_labels.append(-100)
    return new_labels

print(ner_tags)
print(align_only_one_label(ner_tags, inputs.word_ids()))

[3, 0, 7, 0, 0, 0, 7, 0, 0]
[-100, 3, 0, 7, 0, 0, 0, 7, 0, -100, 0, -100]


전체 데이터셋을 전처리하기 위해 모든 입력을 토큰화하고 모든 레이블에 `align_labels_with_tokens()`를 적용해야 한다.

fast tokenizer의 장점을 활용하려면 `batched_true` 옵션을 사용한 `Dataset.map()` 메서드를 사용한다.

In [23]:
# 다중 텍스트(batch)를 입력받아 토큰화하고 레이블 할당하는 함수
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    
    all_labels = examples["ner_tags"]
    new_labels = []
    
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        # new_labels.append(align_labels_with_tokens(labels, word_ids))
        new_labels.append(align_only_one_label(labels, word_ids))
        
    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched = True,
    remove_columns = raw_datasets["train"].column_names
)

print(tokenized_datasets)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})


## Trainer API를 이용한 Fine-Tuning

실제 코드는 3장과 동일하나 batch와 metric 계산 기능만 변경한다.

### Data Collation
앞서 살펴본 `DataCollatorWithPadding`은 입력(input ids, attention mask, token type ids)에 대해서 패딩을 수행하기 때문에 여기서 사용 불가능하다.

여기서는 레이블도 입력과 같은 방식으로 패딩되어야 동일한 크기를 유지하고 -100을 패딩값으로 사용해야 손실 계산 시 무시된다.

따라서 이를 만족하는 ***DataCollatorForTokenClassification*** 을 사용한다.

In [24]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer = tokenizer)

# Test
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
print(len(batch["input_ids"][0]))

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


12


### 평가 기준 (Metrics)

`Trainer`가 매 epoch마다 metrics을 계산하도록 하려면 predictions 및 labels 배열을 입력받아 메트릭 이름과 해당 평가 결과값이 포함된 딕셔너리를 반환하는 `compute_metrics()` 함수를 정의해야 한다. 토큰 분류 예측 평가하는데 많이 사용되는 전통적인 프레임워크는 *seqeval*이다. 이 메트릭을 사용하기 위해 **seqeval** 라이브러리를 먼저 설치해야 한다.(anaconda에서 설치 불가)

설치가 완료되면 `evaluate` 라이브러리의 `load()` 함수를 통해 로드할 수 있다.

In [25]:
from evaluate import load

metric = load("seqeval")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[2023-08-24 09:23:03,213] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


이 메트릭은 우리가 일반적으로 알고있는 표준적인 정확도 accuracy가 아니다. 실제로는 레이블 목록을 정수가 아닌 문자열로 가져오므로 예측 결과와 정답 레이블을 메트릭에 전달하기 전에 디코딩해야 한다.

In [26]:
labels = raw_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [27]:
predictions = labels.copy()
predictions[2] = "O" # 가짜 예측 생성 (Germany의 B-MISC => O)
metric.compute(predictions = [predictions], references = [labels])

{'MISC': {'precision': 1.0,
  'recall': 0.5,
  'f1': 0.6666666666666666,
  'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 0.6666666666666666,
 'overall_f1': 0.8,
 'overall_accuracy': 0.8888888888888888}

위에서 보면 상당히 많은 정보가 출력된다. 전체뿐만 아니라 개별 개체 타입에 대한 정확도, 대현율, F1 점수를 구할 수 있다. 위의 메트릭 계산함수를 수정하여 각자 원하는 점수들을 얻을 수 있도록 수정 가능하다.

아래에 구현된 `compute_metrics()` 함수는 먼저 logits의 argmax를 가져와 predictions으로 변환한다. 늘 그렇듯, logits과 확률이 비례하므로 softmax를 적용할 필요까지는 없다. 그런 다음 레이블과 예측을 정수에서 문자열로 변환해야한다. 레이블이 -100인 모든 값을 제거한 다음 결과를 `metric.compute()` 메서드에 전달한다.

In [28]:
import numpy as np

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    
    # 무시된 인덱스들 제거하고 레이블로 변환
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    all_metrics = metric.compute(predictions = true_predictions, references = true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"]
    }

### 모델 정의하기
`AutoModelForTokenClassification` 모델 사용

이 모델 사용 시 주요 사항은 가지고 있는 레이블 수에 대한 정보를 전달하는 것이다. 이는 `num_labels` 인수를 사용하는 것이지만 올바른 레이블 대응 정보 label correspondences를 설정하는 것이 좋다.

In [29]:
id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label = id2label,
    label2id = label2id
)

# 올바른 수의 레이블이 지정되었는지 확인
model.config.num_labels

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


9

## model fine-tuning

### Training argument와 Trainer를 사용한 Fine-Tuning

In [30]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir = "../../models/7.1/model",
    overwrite_output_dir = True,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = 2e-5,
    num_train_epochs = 3,
    weight_decay = 0.01,
    disable_tqdm = False
)

print(args)

# 드럽게 오래 걸려서 사용자 정의 학습으로 넘어감

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_s

In [31]:
trainer = Trainer(
    model = model,
    args = args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["validation"],
    data_collator = data_collator,
    compute_metrics = compute_metrics,
    tokenizer = tokenizer
)

print(trainer)

trainer.train()

/home/jongg/.virtualenvs/HF_Transformers/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


TrainOutput(global_step=5268, training_loss=0.043871788247209116, metrics={'train_runtime': 436.9696, 'train_samples_per_second': 96.398, 'train_steps_per_second': 12.056, 'total_flos': 921792849708600.0, 'train_loss': 0.043871788247209116, 'epoch': 3.0})

## 사용자 정의 학습 루프

### 학습을 위한 전체 준비

In [32]:
# DataLoader
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle = True,
    collate_fn = data_collator,
    batch_size = 8
)

eval_dataloader = DataLoader(
    tokenized_datasets["validation"],
    collate_fn = data_collator,
    batch_size = 8
)

# Model
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label = id2label,
    label2id = label2id
)

# AdamW optimizer: Adam + Weight decay
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr =  2e-5)

# Accelerator
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


`dataloader`를 `accelerator.prepare()`로 보냈으므로 그 크기를 사용해 training_steps를 계산할 수 있다.

데이터로더를 생성한 후에는 데이터셋의 개수를 변경하기 때문에 항상 아래 작업을 수행해야한다.

(learning_rate가 0까지 줄어드는 고전적인 linear schedule을 사용한다.)

In [33]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer = optimizer,
    num_warmup_steps = 0,
    num_training_steps = num_training_steps
)

### Training loop

In [35]:
# 평가 부분을 단순화하기 위해 metric 객체의 입력을 구성하기 위해서 예측과 레이블을 가져와 문자열 목록으로 변환하는 다음의 postprocess() 함수를 정의
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()
    
    # 무시할 인덱스 제거 후 레이블 변환
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

"""
이제 학습 루프를 작성
학습 진행 방식을 표시하기 위해 progressbar tqdm 사용

루프는 세 부분으로 구성됨:
- training: train_dataloader에서 반복적으로 batch 가져오기, forward pass, backward pass 및 최적화 단계
- evaluation: 하나의 batch에서 모델의 예측 결과를 얻은 후 다음 작업 수행
    - 두 프로세스가 입력과 레이블을 다른 모양으로 padding했을 수 있으므로 gather() 호출 전에 예측과 레이블을 동일한 모양으로 만들기 위해 accelerator.pad_across_processes()를 사용.
    - 결과를 metric.add_batch()로 보내고 평가 루프가 끝나면 metric.compute를 호출
- save: 모델과 토크나이저 저장
"""
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)
        
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
    
    # evaluation
    model.eval()
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)
        
        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]
        
        # 취합 대상인 predictions와 labels를 패딩하기 위해 필요함..
        predictions = accelerator.pad_across_processes(predictions, dim=-1, pad_index = -100)
        labels = accelerator.pad_across_processes(labels, dim=-1, pad_index = -100)
        
        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)
        
        true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions = true_predictions, references = true_labels)
        
    results = metric.compute()
    print(
        f"epoch: {epoch}:",
		{
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
        }
    )
    
    # save
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained("../../models/7.1/model", save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained("../../models/7.1/model")

  0%|          | 0/5268 [00:00<?, ?it/s]

epoch: 0: {'precision': 0.9488387748232918, 'recall': 0.9415497661990648, 'f1': 0.9451802179379716, 'accuracy': 0.9906740391729294}
epoch: 1: {'precision': 0.9498485358465163, 'recall': 0.9422370617696161, 'f1': 0.946027489104928, 'accuracy': 0.9908687356411354}
epoch: 2: {'precision': 0.9498485358465163, 'recall': 0.9422370617696161, 'f1': 0.946027489104928, 'accuracy': 0.9908687356411354}


## Use Fine-Tuned Model

pipeline 사용해 로컬 모델 사용하기

In [36]:
from transformers import pipeline

model_checkpoint = "../../models/7.1/model"
token_classifier = pipeline("token-classification", model = model_checkpoint, aggregation_strategy = "simple")
print(token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn."))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 